In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 99.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00


In [ ]:
!pip install tokenizers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.0 MB/s eta 0:00:00


In [ ]:
import os
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
)
from peft import LoraConfig, PeftModel

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**INFERENCE**

In [ ]:
new_model = "/content/drive/MyDrive/ml/llama-2-7b-supercode"
model_name = "NousResearch/Llama-2-7b-chat-hf"

In [ ]:
m = AutoModelForCausalLM.from_pretrained(
    model_name,
    #load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map={"": 0}
)


In [ ]:
m = PeftModel.from_pretrained(m,new_model)
m = m.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
import pandas as pd
import tqdm

csv_filepath = './ml/inputs3.csv'
df = pd.read_csv(csv_filepath)
pipe = pipeline(task="text-generation", model=m, tokenizer=tokenizer, max_length=1024, temperature=0.7, return_full_text=False)

results = []

for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0]):
    task = row['task']
    data = row['data_type']
    metric = row['metrics']
    data_card = row['data_card']
    submission = row['submission']
    link = row['link']

    options = {}
    for place, option in zip(['first', 'second', 'third'], ['option_1', 'option_2', 'option_3']):
        prompt = f'''[INST] <<SYS>>
        You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
        <</SYS>>
        Imagine that you are a data analyst. Your objective is writing the {place} place instruction for solving this machine learning task. Task: {task}
        The {data} data is used for the problem. The metric type is {metric} for the problem. Your response contains the main information about data preprocessing, model architecture and model training.''[/INST]'''


        generated_text = pipe(prompt)[0]['generated_text']
        options[option] = generated_text


    results.append({
        'task': task,
        'data_type': data,
        'metric': metric,
        'data_card': data_card,
        'option_1': options['option_1'],
        'option_2': options['option_2'],
        'option_3': options['option_3'],
        'submission': submission,
        'link': link
    })


results_df = pd.DataFrame(results)
results_df.to_csv('./ml/results3.csv', index=False)

In [ ]:
print(result[0]['generated_text'])

[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
Imagine that you are is an data analyst. Your objective is writing the 1st best instruction for solving this machine learning task. Task: You are tasked with developing a model to predict the closing price movements of Nasdaq listed stocks using data from the order book and the closing auction. The goal is to enhance market efficiency, especially during the final ten minutes of trading, similar to challenges faced by firms like Optiver.The tabular data is used for